# Language Agent Tree Search

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/agent/lats_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[LATS (Language Agent Tree Search)](https://arxiv.org/pdf/2310.04406.pdf) by Zhou et al. combines LLM capabilities in planning, acting, and reasoning within a Monte Carlo tree search framework, allowing for deliberate and adaptive problem-solving guided by external feedback and self-reflection.

We've implemented this agent as a LlamaPack - you can either pip install it to run it out-of-the-box or call `download_llama_pack` to load the pack.

## Setup

In [ ]:
%pip install llama-index-agent-lats
%pip install llama-index-program-openai
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-core llama-index-readers-file

### Define Global Settings

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

# NOTE: a higher temperate will help make the tree-expansion more diverse
llm = OpenAI(model="gpt-4-turbo", temperature=0.6)
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

Settings.llm = llm
Settings.embed_model = embed_model

### Download Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [ ]:
import os
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.core.storage import StorageContext


if not os.path.exists("./storage/lyft"):
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")
else:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

### Setup Tools

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool. "
                "The input is used to power a semantic search engine."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool. "
                "The input is used to power a semantic search engine."
            ),
        ),
    ),
]

## Set up Agent

Now we can set up the LATS agent.

Here, `num_expansions` refers to the number of possible sub-actions to explore under each node. `num_expansions=2` means we will explore to possible next-actions for every parent action.

`max_rollouts` refers to how deep each exploration of the search space continues. `max_rollouts=5` means a maximum depth of 5 is explored in the tree.

In [ ]:
from llama_index.agent.lats import LATSAgentWorker

agent_worker = LATSAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    num_expansions=2,
    max_rollouts=3,  # using -1 for unlimited rollouts
    verbose=True,
)
agent = agent_worker.as_agent()

## Run Some Queries

First, lets use step-wise execution and the lower-level API to create and execute a task.

In [ ]:
task = agent.create_task(
    "Given the risk factors of Uber and Lyft described in their 10K files, "
    "which company is performing better? Please use concrete numbers to inform your decision."
)

In [ ]:
# run initial step
step_output = agent.run_step(task.task_id)

> Selecting node to expand: Observation: Given the risk factors of Uber and Lyft described in their 10K files, which company is performing better? Please use concrete numbers to inform your decision.
> Got candidates: ['Retrieve and compare the latest financial metrics from the 10K files of Uber and Lyft, such as revenue growth, profit margins, and debt levels.', 'Analyze the specific risk factors listed in the 10K files for Uber and Lyft and determine which company has a more manageable risk profile.']
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What is the revenue growth, profit margins, and debt levels for Uber in 2021?"}
=== Calling Function ===
Calling function: lyft_10k with args: {"input": "What are the main risk factors listed in the Lyft 10K report for 2021?"}
=== Function Output ===
In 2021, Uber's revenue grew by 57%, increasing from $11,139 million in 2020 to $17,455 million. The company's loss from operations improved, showing a decrease from 

From the step output, we can inspect the task state

In [ ]:
for step in (
    step_output.task_step.step_state["root_node"].children[0].current_reasoning
):
    print(step)
    print("---------")

observation='Given the risk factors of Uber and Lyft described in their 10K files, which company is performing better? Please use concrete numbers to inform your decision.' return_direct=False
---------
observation='Review the 10K files of Uber and Lyft to extract relevant financial data and risk factors.' return_direct=False
---------


In [ ]:
for step in (
    step_output.task_step.step_state["root_node"]
    .children[0]
    .children[0]
    .current_reasoning
):
    print(step)
    print("---------")

observation='Given the risk factors of Uber and Lyft described in their 10K files, which company is performing better? Please use concrete numbers to inform your decision.' return_direct=False
---------
observation='Review the 10K files of Uber and Lyft to extract relevant financial data and risk factors.' return_direct=False
---------
thought='To compare the performance of Uber and Lyft using their 10K files, I need to gather financial data and risk factors from both companies. I will start by querying the Uber 10K tool.' action='uber_10k' action_input={'input': 'What are the key financial figures and risk factors for Uber in 2021?'}
---------
observation="Uber Technologies, Inc., in its 2021 financial statements, consolidates its wholly-owned and majority-owned subsidiaries, as well as variable interest entities where it is the primary beneficiary. The financial statements are prepared in accordance with GAAP, and management uses estimates and assumptions that affect reported financi

Lets loop until we finish the task

In [ ]:
# repeat until the last step is reached
while not step_output.is_last:
    step_output = agent.run_step(task.task_id)

response = agent.finalize_response(task.task_id)

> Selecting node to expand: Observation: Uber Technologies, Inc., in its 2021 financial statements, consolidates its wholly-owned and majority-owned subsidiaries, as well as variable interest entities where it is the primary beneficiary. The financial statements are prepared in accordance with GAAP, and management uses estimates and assumptions that affect reported financial figures, such as the fair values of investments, useful lives of assets, and reserves for income taxes and insurance, among others. These estimates consider the impact of the COVID-19 pandemic on market data and investment recoverability.

Key financial risks for Uber include concentration of credit risk, where cash and other receivables are potentially subject to credit risk concentration. The company's cash, cash equivalents, and securities consist largely of high-credit-quality money market funds, U.S. government and agency securities, and corporate debt securities. Despite exceeding insured limits, these are pl

In [ ]:
print(str(response))

Based on the information provided from the 10K files of Uber and Lyft for 2021, here is a comparative analysis of their financial data and risk factors:

**Financial Data:**
1. **Uber** did not disclose specific revenue figures in the provided information, focusing more on their financial management practices and the quality of their financial instruments. They emphasized the high credit quality of their cash equivalents and securities, and the use of GAAP-compliant financial statements influenced by estimates considering the COVID-19 impact.
   
2. **Lyft** reported total revenue of $3,208,323,000 for 2021, with a significant drop due to the COVID-19 pandemic and a partial recovery, still below pre-pandemic levels. This gives a clear numeric insight into their financial status during the year.

**Risk Factors:**
1. **Uber's** key risk factors include concentration of credit risk, with their financial assets placed in high-credit-quality institutions, and various uncertainties that cou

Using `.chat()` directly, we can have the agent automatically run until completion.

In [ ]:
agent.reset()

response = agent.chat(
    "Given the revenue growth and risk factors of Uber and Lyft, "
    "which company is performing better? Please use concrete numbers to inform your decision."
)

> Selecting node to expand: Observation: Given the revenue growth and risk factors of Uber and Lyft, which company is performing better? Please use concrete numbers to inform your decision.
> Got candidates: ['Review financial reports of Uber and Lyft for the latest fiscal year to compare revenue growth.', 'Analyze risk factors mentioned in the latest quarterly reports of Uber and Lyft.']
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What was Uber's revenue growth for the fiscal year 2021?"}
=== Function Output ===
Uber's revenue for the fiscal year 2021 was $17,455 million, which represents a growth of 57% compared to the revenue of $11,139 million in 2020.
> Generated new reasoning step: Thought: The current language of the user is English. I need to use tools to gather data on the revenue growth of Uber and Lyft for the latest fiscal year, which is 2021.
Action: uber_10k
Action Input: {'input': "What was Uber's revenue growth for the fiscal year 2021?"}
O

In [ ]:
print(str(response))

Uber's revenue growth for the fiscal year 2021 was 57%, increasing from $11,139 million in 2020 to $17,455 million in 2021. Lyft's revenue growth for the same period was 36%. Comparing these figures, Uber has shown a higher revenue growth percentage than Lyft for the fiscal year 2021.
